<a href="https://colab.research.google.com/github/Vishwaak/Time-for-Machine/blob/master/Differential_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

num_entries = 5000
db = torch.rand(num_entries) > 0.5
db


tensor([0, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8)

In [0]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [0]:
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs
  

In [0]:
pdbs = get_parallel_dbs(db)

In [0]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs


In [0]:


db, pdbs = create_db_and_parallels(5000)



In [0]:
def query(db):
  return db.sum()

In [0]:
full_db_result = query(db)

In [0]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [10]:
sensitivity

tensor(1)

In [0]:
def sensitivity(query, n_entries=1000):

    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > max_distance):
            max_distance = db_distance
            
    return max_distance
    
    

In [0]:
def query(db):
    return db.float().mean()

In [13]:
sensitivity(query)

tensor(0.0005)

In [0]:
db, pdbs = create_db_and_parallels(20)

In [15]:
db


tensor([0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0],
       dtype=torch.uint8)

In [0]:
def query(db, threshold=5):
    return (db.sum() > threshold).float()

In [17]:
for i in range(10):
    sens_f = sensitivity(query, n_entries=10)
    print(sens_f)


0
0
tensor(1.)
0
0
0
0
0
0
0


In [0]:

db, _  = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db , remove_index=10)

In [22]:
db[10]

tensor(0, dtype=torch.uint8)

In [23]:
sum(db)

tensor(48, dtype=torch.uint8)

In [24]:
sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [25]:
(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0048)

In [26]:
(sum(db).float() > 49) - (sum(pdb).float()  > 49)

tensor(0, dtype=torch.uint8)